# run_r usage examples as tests

Each code cell below mirrors a usage example from `usage_examples.py`, using separate R scripts under `r_scripts/`.

In [1]:
import os, sys
# add .. to path to allow imports from parent directory
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
from run_r import run_r_script, RScriptRunner

try:
    import pandas as pd
    import numpy as np
except ImportError as e:
    print("Optional dependencies missing:", e)

print("Environment ready.")

Environment ready.


Test 1: Basic usage (no input data) – runs `example.R`.

In [2]:
variables = run_r_script("r_scripts/example.R", verbose=False)
assert isinstance(variables, dict)
assert 'x' in variables
assert 'result' in variables
print("Example 1 passed.", list(variables.keys()))

Using R executable: C:\Users\garciaba\AppData\Local\Programs\R\R-4.5.1\bin\Rscript.exe
Example 1 passed. ['colors', 'data', 'is_active', 'my_list', 'my_matrix', 'name', 'numbers', 'pi_value', 'result', 'x']


Test 2: Basic Python types to R – uses `r_scripts/basic_types.R`.

In [3]:
input_data = {
    "name": "Alice",
    "age": 25,
    "is_student": True,
    "scores": [85, 90, 92, 88]
}
vars2 = run_r_script("r_scripts/basic_types.R", input_data=input_data, verbose=False)
assert 'result' in vars2
assert 'message' in vars2
print("Example 2 passed.", vars2['message'])

Using R executable: C:\Users\garciaba\AppData\Local\Programs\R\R-4.5.1\bin\Rscript.exe
Example 2 passed. Hello Alice - average score: 88.75


Test 3: Pandas DataFrame to R – uses `r_scripts/dataframe.R`.

In [4]:
df = pd.DataFrame({
    'product': ['Widget A', 'Widget B', 'Widget C', 'Widget A', 'Widget B'],
    'amount': [100.50, 250.75, 150.25, 300.00, 175.50],
    'quantity': [2, 5, 3, 6, 4]
})
vars3 = run_r_script("r_scripts/dataframe.R", input_data={"sales_data": df}, verbose=False)
assert 'total_revenue' in vars3
assert 'avg_amount' in vars3
assert 'top_product' in vars3
assert 'summary_stats' in vars3
print("Example 3 passed.")

# display the dataframe summary statistic
vars3['summary_stats']

Using R executable: C:\Users\garciaba\AppData\Local\Programs\R\R-4.5.1\bin\Rscript.exe
Example 3 passed.


,metric,value
1,Total Revenue,977
2,Average Amount,195.4
3,Top Product,Widget A


Test 4: Multiple inputs (DataFrame + parameters) – uses `r_scripts/multiple_inputs.R`.

In [5]:
df = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'value': [10, 25, 30, 15, 40, 5],
    'category': ['A', 'B', 'A', 'C', 'B', 'A']
})
input_data = {"data": df, "threshold": 20, "multiplier": 1.5}
vars4 = run_r_script("r_scripts/multiple_inputs.R", input_data=input_data, verbose=False)
assert 'mean_adjusted' in vars4
assert 'total_adjusted' in vars4
assert 'result_df' in vars4 and vars4['result_df'] is not None
print("Example 4 passed.", vars4['mean_adjusted'], vars4['total_adjusted'])

Using R executable: C:\Users\garciaba\AppData\Local\Programs\R\R-4.5.1\bin\Rscript.exe
Example 4 passed. 47.5 142.5


Test 5: Reusable RScriptRunner – uses `r_scripts/sum_script.R` and `r_scripts/product_script.R`.

In [6]:
runner = RScriptRunner()
res1 = runner.run_script("r_scripts/sum_script.R", verbose=False, debug=False)
res2 = runner.run_script("r_scripts/product_script.R", verbose=False, debug=False)
assert 'sum_result' in res1
assert 'product_result' in res2
print("Example 5 passed.", res1['sum_result'], res2['product_result'])

Using R executable: C:\Users\garciaba\AppData\Local\Programs\R\R-4.5.1\bin\Rscript.exe
Example 5 passed. 30 15


Test 6: Numpy arrays to R – uses `r_scripts/numpy_arrays.R`.

In [7]:
import numpy as np
vector = np.array([1.5, np.nan, 3.9, 4.2, 5.1])
matrix = np.array([[1, 2, 3], [np.nan, 5, 6]])
vars6 = run_r_script("r_scripts/numpy_arrays.R", input_data={"vector": vector, "matrix": matrix}, verbose=False)
assert 'mean_vector' in vars6
assert 'sum_matrix' in vars6
assert 'max_vector' in vars6
assert 'squared' in vars6
print("Example 6 passed.", vars6['mean_vector'], vars6['sum_matrix'], vars6['max_vector'])

Using R executable: C:\Users\garciaba\AppData\Local\Programs\R\R-4.5.1\bin\Rscript.exe
Example 6 passed. 3.675 17 5.1


Test 7: Statistical models with lme4 – uses `r_scripts/models_glmer.R` (may install packages and take longer).

In [8]:
import numpy as np
import pandas as pd
np.random.seed(42)
n_subjects = 20
n_obs_per_subject = 30
subject_ids = np.repeat(range(n_subjects), n_obs_per_subject)
subject_effects = np.random.randn(n_subjects) * 0.5
predictor = np.random.randn(n_subjects * n_obs_per_subject)
linear_pred = 0.5 * predictor + subject_effects[subject_ids]
prob = 1 / (1 + np.exp(-linear_pred))
response = (np.random.rand(len(prob)) < prob).astype(int)
df = pd.DataFrame({
    'subject_id': subject_ids,
    'predictor': predictor,
    'response': response
})
try:
    vars7 = run_r_script("r_scripts/models_glmer.R", input_data={"study_data": df}, verbose=False, debug=False)
    assert 'model' in vars7 or 'fixed_effects' in vars7 or 'fit_stats' in vars7
    print("Example 7 passed.", list(vars7.keys()))
except Exception as e:
    print("Example 7 skipped due to error:", e)



Using R executable: C:\Users\garciaba\AppData\Local\Programs\R\R-4.5.1\bin\Rscript.exe
Example 7 passed. ['fit_stats', 'fixed_effects', 'model', 'random_effects', 'study_data']
